# Dockerize Notebook
The purpose of this notebook is to aid my tutorial found [here](https://medium.com/@patrickhanbury1/how-to-launch-docker-containers-with-the-official-python-library-31dbe1bff8ca). This notebook is intended to function as a standlone piece that both clones the rHuggler's Django repository for you and launches the Docker containers.

In [1]:
import docker
import os
from git import Repo
import shutil

In [2]:
client = docker.from_env()
cwd = os.getcwd()
# dock_pth = os.path.join(cwd,'todo')

## Clone Repo

In [3]:
git_url = 'https://github.com/rHuggler/django-todo.git'
repo_dir = os.path.join(cwd, 'todo')
if os.path.isdir(repo_dir):
    print(f"Removing {repo_dir}")
    shutil.rmtree(repo_dir, ignore_errors=True)
Repo.clone_from(git_url, repo_dir);

## Clean Up Docker
This was not included in the tutorial, but might be helpful if you want to automate the pruning and removal of your Docker objects

In [4]:
# remove dangling containers
db_name = 'db'
web_name = 'web'
api_name = 'api'
names = [db_name, web_name, api_name]
for cont in client.containers.list(all=True):
    if cont.name in names:
        cont.stop() # if running
        try:
            cont.remove()
        except:
            pass

In [5]:
# remove any dangling nets
net_name = 'djang-net'
nets = client.networks.list()
for net in nets:
    if net.name == net_name:
        net.remove()

In [6]:
## Create Shell Script and Move to Todo
with open('todo/api/django_todo/api.sh', 'w') as fb:
    cmds = ['python manage.py makemigrations', 'python manage.py migrate',
           'python manage.py runserver 0.0.0.0:8000']
    for cmd in cmds:
        fb.write(cmd + '\n')
        
with open('todo/web/vue-todo/web.sh', 'w') as fb:
    cmds = ['npm install', 'npm run dev']
    for cmd in cmds:
        fb.write(cmd + '\n')

## Build Images

In [7]:
web_pth = os.path.join(repo_dir, 'web')
web_img, logs = client.images.build(path=web_pth, tag='web')

In [8]:
api_pth = os.path.join(repo_dir, 'api')
api_img, logs = client.images.build(path=api_pth, tag='api')

## Build Bridge Network

In [9]:
net = client.networks.create(name=net_name, driver='bridge')

## Run Containers

### Define customizable parameters

In [10]:
# ports
api_ports = {'8000/tcp' : 8000}
web_ports = {'8080/tcp' : 8080}
web_env = ["HOST=0.0.0.0"]

# volumes
api_vol = { 
    os.path.join(repo_dir, 'api/django_todo') : {'bind': '/usr/src/app', 'mode': 'rw'}
}

web_vol = {
    os.path.join(repo_dir, 'web/vue-todo') : { 'bind' : '/usr/src/app', 'mode' : 'rw'}
}

db_vol = {
    os.path.join(repo_dir, 'dbdata') : {'bind' : '/var/lib/postgresql/data'}
}

### Launch Containers

In [11]:
db = client.containers.run('postgres:10', detach=True, name='db', 
                           network=net_name, volumes=db_vol)

api = client.containers.run(api_img, ports=api_ports, detach=True, 
                            name='api', network=net_name, volumes=api_vol,
                            command="sh api.sh")

# api.restart()

web = client.containers.run(web_img, ports=web_ports, detach=True,
                           name='web', network=net_name, volumes=web_vol,
                           environment=web_env, command="sh web.sh")